<a href="https://colab.research.google.com/github/Vishnu-bhat/YoloV8/blob/main/GOT_OUTPUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Aug  8 14:47:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import torch

# Check PyTorch GPU availability
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Is Available: {torch.cuda.is_available()}")
else:
    print("GPU is not available.")

GPU Name: Tesla T4
GPU Is Available: True


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [2]:
%cd /mydrive/Yolo

/content/gdrive/My Drive/Yolo


In [6]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [7]:
!pip install opencv-python

In [8]:
!pip install cvzone

In [9]:
from ultralytics import YOLO

In [10]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
          # print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [3]:
import cv2
import pandas as pd
import cvzone
import numpy as np
import torch

In [11]:
model=YOLO('best.pt')

In [12]:
# def RGB(event, x, y, flags, param):
#    if event == cv2.EVENT_MOUSEMOVE :
#       point = [x, y]
#        print(point)

In [ ]:
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

In [4]:
cap=cv2.VideoCapture('/mydrive/Yolo/id4.mp4')

In [ ]:
my_file = open("coco1.txt", "r")
data = my_file.read()

In [18]:
class_list = data.split("\n")
print(class_list)

['motor-cycle', 'car', 'auto-rikshaw', 'bus']


In [21]:
cy1=427
offset=6

count=0
tracker=Tracker()
tracker1=Tracker()
tracker2=Tracker()
tracker3=Tracker()

bus=[]
car=[]
auto_rikshaw=[]
motorcycle=[]

In [16]:
while True:
    ret,frame = cap.read()
    if not ret:
       break

    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    results=model.predict(frame)
    # print(results)



    # Assuming `results[0].boxes.data` is a tensor
    a = results[0].boxes.data

    # Move the tensor to CPU (if it's on GPU) and convert to NumPy
    a_cpu = a.cpu().numpy()

    # Convert the NumPy array to a Pandas DataFrame and ensure it's of type float
    px = pd.DataFrame(a_cpu).astype("float")

    # px=pd.DataFrame(a).astype("float")

    # print(px)
    list=[]
    list1=[]
    list2=[]
    list3=[]
    for index,row in px.iterrows():
    # print(row)

        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])

        d=int(row[5])
        c=class_list[d]

        if 'bus' in c:
            list.append([x1,y1,x2,y2])
        elif 'car' in c:
             list1.append([x1,y1,x2,y2])
        elif 'auto-rikshaw' in c:
             list2.append([x1,y1,x2,y2])
        elif 'motor-cycle' in c:
             list3.append([x1,y1,x2,y2])

    bbox_idx=tracker.update(list)
    bbox1_idx=tracker1.update(list1)
    bbox2_idx=tracker2.update(list2)
    bbox3_idx=tracker3.update(list3)

    for bbox in bbox_idx:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        if cy1<(cy+offset) and cy1>(cy-offset):
           cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
           cvzone.putTextRect(frame,f'{id}',(x3,y3),1,1)
           if bus.count(id)==0:
              bus.append(id)
#####################################CAR#################################
    for bbox1 in bbox1_idx:
        x5,y5,x6,y6,id1=bbox1
        cx2=int(x5+x6)//2
        cy2=int(y5+y6)//2
        if cy1<(cy2+offset) and cy1>(cy2-offset):
           cv2.rectangle(frame,(x5,y5),(x6,y6),(0,255,0),2)
           cvzone.putTextRect(frame,f'{id1}',(x5,y5),1,1)
           if car.count(id1)==0:
              car.append(id1)
#################################auto-rikshaw############################
    for bbox2 in bbox2_idx:
        x7,y7,x8,y8,id2=bbox2
        cx3=int(x7+x8)//2
        cy3=int(y7+y8)//2
        if cy1<(cy3+offset) and cy1>(cy3-offset):
           cv2.rectangle(frame,(x7,y7),(x8,y8),(0,255,0),2)
           cvzone.putTextRect(frame,f'{id2}',(x7,y7),1,1)
           if auto_rikshaw.count(id2)==0:
              auto_rikshaw.append(id2)
#########################################motorcycle##############################
    for bbox3 in bbox3_idx:
        x9,y9,x10,y10,id3=bbox3
        cx4=int(x9+x10)//2
        cy4=int(y9+y10)//2
        if cy1<(cy4+offset) and cy1>(cy4-offset):
           cv2.rectangle(frame,(x9,y9),(x10,y10),(0,255,0),2)
           cvzone.putTextRect(frame,f'{id3}',(x9,y9),1,1)
           if motorcycle.count(id3)==0:
              motorcycle.append(id3)

    countbus=(len(bus))
    countcar=(len(car))
    countauto_rikshaw=(len(auto_rikshaw))
    countmotorcycle=(len(motorcycle))

    print(len(bus))
    print(len(car))
    print(len(auto_rikshaw))
    print(len(motorcycle))

    cvzone.putTextRect(frame,f'buscount:-{countbus}',(50,60),2,2)
    cvzone.putTextRect(frame,f'countcar:-{countcar}',(50,140),2,2)
    cvzone.putTextRect(frame,f'countauto_rikshaw:-{countauto_rikshaw}',(600,60),2,2)
    cvzone.putTextRect(frame,f'countmotorcycle:-{countmotorcycle}',(600,140),2,2)

    cv2.line(frame,(355,427),(568,427),(255,255,255),2)
    #cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
0: 416x800 (no detections), 11.5ms
Speed: 3.1ms preprocess, 11.5ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 8.8ms
Speed: 6.4ms preprocess, 8.8ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 12.1ms
Speed: 3.1ms preprocess, 12.1ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 12.2ms
Speed: 9.1ms preprocess, 12.2ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 14.6ms
Speed: 3.4ms preprocess, 14.6ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 14.8ms
Speed: 3.3ms preprocess, 14.8ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 (no detections), 10.7ms
Speed: 3.1ms preprocess, 10.7ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 800)

0: 416x800 1 motorcy

In [19]:
print(len(bus))
print(len(car))
print(len(auto_rikshaw))
print(len(motorcycle))

2
10
17
99
